# OTROS TIEMPOS

In [1]:
weeks = 52

In [2]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)
# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.operarios.tiempos", "search",
    [[["inicio", ">=", FECHA_INICIO], ["inicio", "<=", FECHA_FIN]]]
)

lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.operarios.tiempos", "read",
    [lost_sales_ids],
    {"fields": []}
)



#"display_name","inicio","finalizo","is_pause"

df = pd.DataFrame(lost_sales_data)

df['tarea'] = df['tarea_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

#Tarda  menos de 1 minuto

📅 Descargando registros desde 2024-10-28 hasta 2025-10-27


In [3]:
df.head()

,create_uid,__last_update,wk_line_id,create_date,name,tipo,tarea_id,delay_pausa,is_pause,finalizo,delay_parcial,delay_total,production_id,inicio,write_date,workcenter_id,costo_total,write_uid,id,delay_hours,display_name,tarea
0,"[345, Planta1]",2024-10-29 13:01:15,"[62771, PLISADORA 1 - DA4702 FILTRO AIRE 2_ CHEV FVR, IHC.]",2024-10-28 11:31:45,"[355, DANILO PUPO SIERRA]",sistema,"[2, Produccion]",86277.0,False,2024-10-29 08:01:15,37853.0,37853.0,"[11495, MO11448]",2024-10-28 06:31:45,2024-10-29 13:01:15,"[56, PLISADO]",23557.97,"[345, Planta1]",1895,10.51,DANILO PUPO SIERRA,Produccion
1,"[345, Planta1]",2024-10-28 12:45:08,"[62363, EMBALAJE - DA4570 FILTRO AIRE 2. DONSSON - PERKINS, CATERPILLAR.]",2024-10-28 11:56:22,"[603, INGRID CAROLINA GOMEZ DELPRADO ]",sistema,"[2, Produccion]",2922.0,False,2024-10-28 07:45:08,2924.0,2924.0,"[11443, MO11395]",2024-10-28 06:56:23,2024-10-28 12:45:08,"[57, EMBALAJE]",11.98,"[345, Planta1]",1896,0.81,INGRID CAROLINA GOMEZ DELPRADO,Produccion
2,"[345, Planta1]",2024-10-28 12:34:10,False,2024-10-28 11:56:44,"[516, ERIKA YULIEDT UMAÑA SABOGAL]",produccion,"[9, Reproceso]",0.0,False,2024-10-28 07:34:10,0.0,0.0,False,2024-10-28 06:56:44,2024-10-28 12:34:10,False,8392.83,"[345, Planta1]",1897,0.62,ERIKA YULIEDT UMAÑA SABOGAL,Reproceso
3,"[345, Planta1]",2024-10-29 13:00:22,"[62771, PLISADORA 1 - DA4702 FILTRO AIRE 2_ CHEV FVR, IHC.]",2024-10-28 11:59:13,"[642, BLANCA CECILIA AVELLA ARIAS]",sistema,"[2, Produccion]",85892.0,False,2024-10-29 08:00:22,34889.0,34889.0,"[11495, MO11448]",2024-10-28 06:59:13,2024-10-29 13:00:22,"[56, PLISADO]",14792.51,"[345, Planta1]",1898,9.69,BLANCA CECILIA AVELLA ARIAS,Produccion
4,"[345, Planta1]",2024-10-28 20:31:05,False,2024-10-28 12:00:19,"[553, KATHERIN ANDREA GIL AGUILAR]",produccion,"[12, Auxiliar producción]",0.0,False,2024-10-28 15:31:08,0.0,0.0,False,2024-10-28 07:00:19,2024-10-28 20:31:05,False,118821.98,"[1, Administrador Administrador ]",1899,8.51,KATHERIN ANDREA GIL AGUILAR,Auxiliar producción


In [4]:
df_ot = df[df["tipo"]!="sistema"] #Se quitan las de op ya que no caen bien

df_ot.columns = df_ot.columns.str.lower()
df_ot = df_ot.rename(columns={
    "display_name": "operario",
    "inicio": "fecha_inicio",
    "tipo": "actividad",
})

# Convertir fechas
for col in ["fecha_inicio", "create_date", "__last_update"]:
    df_ot[col] = pd.to_datetime(df_ot[col], errors="coerce")

# Agregar columnas que no existen en df_ot para igualar estructura
df_ot["duracion_horas"] = df_ot["delay_hours"]
df_ot["centro_produccion"] = "OTRAS TAREAS"

df_ot = df_ot[["operario","__last_update","actividad","finalizo","fecha_inicio","tarea","duracion_horas"]]

# Hora actual (en tu zona horaria si quieres)
ahora = pd.Timestamp.now()

df_ot["fecha_inicio"] = pd.to_datetime(df_ot["fecha_inicio"], errors="coerce")
df_ot["finalizo"] = pd.to_datetime(df_ot["finalizo"], errors="coerce")

# --- 1️⃣ Crear columna en_curso ---
# True si no hay fecha de finalización
df_ot["en_curso"] = df_ot["finalizo"].isna() | (df_ot["finalizo"] == False)

# --- 2️⃣ Recalcular duración ---
df_ot["duracion_horas"] = df_ot.apply(
    lambda row: (
        (ahora - row["fecha_inicio"]).total_seconds() / 3600
        if row["en_curso"] and pd.notna(row["fecha_inicio"])
        else (row["finalizo"] - row["fecha_inicio"]).total_seconds() / 3600
        if pd.notna(row["finalizo"]) and pd.notna(row["fecha_inicio"])
        else 0
    ),
    axis=1
).round(2)

In [5]:
df_ot.tail(20)

,operario,__last_update,actividad,finalizo,fecha_inicio,tarea,duracion_horas,en_curso
32552,ZORAIDA CHAPARRO ALVAREZ,2025-10-24 13:10:47,produccion,2025-10-24 08:10:48,2025-10-24 07:59:36,Reproceso,0.19,False
32562,MARIA HELENA DELPRADO PEREZ,2025-10-24 15:19:22,produccion,2025-10-24 10:19:22,2025-10-24 08:31:47,Entrega producto terminado,1.79,False
32577,KATHERIN ANDREA GIL AGUILAR,2025-10-24 15:53:57,produccion,2025-10-24 10:53:57,2025-10-24 10:00:58,Auxiliar producción,0.88,False
32599,INGRID CAROLINA GOMEZ DELPRADO,2025-10-24 18:02:50,produccion,2025-10-24 13:02:52,2025-10-24 12:31:07,Reproceso,0.53,False
32630,ZORAIDA CHAPARRO ALVAREZ,2025-10-24 20:01:22,produccion,2025-10-24 15:01:22,2025-10-24 15:01:22,Orden y Aseo,0.00,False
32633,KATHERIN ANDREA GIL AGUILAR,2025-10-27 13:16:42,produccion,2025-10-27 08:16:42,2025-10-27 06:54:14,Auxiliar producción,1.37,False
32634,JUAN DAVID GUTIERREZ CORTES,2025-10-27 11:54:21,produccion,NaT,2025-10-27 06:54:21,Molderia,8.41,True
32635,DAVID ESTEBAN MALAGON CETINA,2025-10-27 14:42:51,produccion,2025-10-27 09:42:51,2025-10-27 06:56:07,Alistamiento MP,2.78,False
32639,YENNY KATERINE PATAQUIVA NEUSA,2025-10-27 16:33:29,produccion,2025-10-27 11:33:29,2025-10-27 07:04:41,Permiso,4.48,False
32640,MARIA PAULA PARRA PINILLA,2025-10-27 14:43:05,produccion,2025-10-27 09:43:05,2025-10-27 07:05:36,Reproceso,2.62,False


In [6]:
#df_ot.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/otras_tareas.xlsx")

# TIEMPOS POR OP

In [7]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd
import re

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)
# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

ops_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.tiempos.trabajo", "search",
    [[["create_date", ">=", FECHA_INICIO], ["create_date", "<=", FECHA_FIN]]]
)

t_op = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.tiempos.trabajo", "read",
    [ops_ids],
    {"fields": []}
)



#"display_name","inicio","finalizo","is_pause"

df_op = pd.DataFrame(t_op)

df_op['Display_name'] = df_op['operario'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

df_op['OP'] = df_op['production_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

df_op['centro_produccion'] = df_op['workcenter_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))


df_op["nom_filtro"] = df_op["product_fabricado_id"].astype(str).apply(
    lambda x: re.search(r"\(([^)]+)\)", x).group(1) if re.search(r"\(([^)]+)\)", x) else None
)

df_op["product_ref"] = df_op["product_fabricado_id"].astype(str).apply(
    lambda x: re.search(r"\[([A-Z0-9]+)\]", x).group(1) if re.search(r"\[([A-Z0-9]+)\]", x) else None
)

df_op["maquina"] = df_op["fuerza_lab_id"].astype(str).apply(
    lambda x: re.search(r",\s*[\'\"]?([A-ZÁÉÍÓÚÑ0-9\s]+?)(?=\s*-\s*)", x, re.IGNORECASE).group(1).strip()
    if re.search(r",\s*[\'\"]?([A-ZÁÉÍÓÚÑ0-9\s]+?)(?=\s*-\s*)", x, re.IGNORECASE)
    else None
)



df_op = df_op.drop(columns=["operario", "production_id", "workcenter_id","write_uid","id","display_name","create_uid","wc_line_id","product_fabricado_id"])

#Tarda aprox 4 minutos


📅 Descargando registros desde 2024-10-28 hasta 2025-10-27


In [8]:
# --- Limpieza df_op ---
df_op.columns = df_op.columns.str.lower()
df_op = df_op.rename(columns={"evento": "actividad", "display_name": "operario"})

# Asegurar tipos
for col in ["fecha_evento", "create_date", "write_date", "__last_update"]:
    df_op[col] = pd.to_datetime(df_op[col], errors="coerce")


In [9]:
from datetime import timedelta

# Asegurar tipos de fecha correctos
df_op["fecha_evento"] = pd.to_datetime(df_op["fecha_evento"], errors="coerce")

# --- 1️⃣ Crear columna 'final' ---
df_op["final"] = df_op.apply(
    lambda row: row["fecha_evento"] + timedelta(seconds=row["segundos_evento"])
    if pd.notna(row["segundos_evento"]) else row["fecha_evento"],
    axis=1
)

# --- 2️⃣ Ordenar por operario y fecha_evento ---
df_op = df_op.sort_values(["operario", "fecha_evento"])

# --- 3️⃣ Marcar el último evento por operario ---
df_op["ultimo_evento"] = df_op.groupby("operario")["fecha_evento"].transform("max")

# --- 4️⃣ Crear columna en_curso ---
# En curso = el evento más reciente del operario y su actividad es "iniciar"
df_op["en_curso"] = (df_op["fecha_evento"] == df_op["ultimo_evento"]) & (df_op["actividad"] == "iniciar")

# --- 5️⃣ Filtrar: eliminar 'iniciar' antiguos (solo mantener el último 'iniciar' si está en curso) ---
df_op_filtrado = df_op[
    (df_op["actividad"] != "iniciar") |  # mantener todo lo que no sea iniciar
    (df_op["en_curso"])                  # o mantener el último iniciar de cada operario
].copy()

# --- 6️⃣ Limpieza final ---
df_op_filtrado = df_op_filtrado.drop(columns=["ultimo_evento"])


In [13]:
df_op_filtrado[df_op_filtrado["en_curso"]==True]

,__last_update,horas_evento,create_date,fecha_evento,fuerza_lab_id,actividad,segundos_evento,write_date,operario,op,centro_produccion,nom_filtro,product_ref,maquina,final,en_curso
98089,2025-10-27 20:12:54,0.0,2025-10-27 20:12:54,2025-10-27 15:12:54,"[114339, TAPAS - DA9022 FILTRO AIRE DONSSON - LOVOL]",iniciar,0.0,2025-10-27 20:12:54,ALVARO MELO PRADA,MO13355,CORTE,025 DA9022,DAE09022025,TAPAS,2025-10-27 15:12:54,True
98083,2025-10-27 20:01:17,0.0,2025-10-27 20:01:17,2025-10-27 15:01:17,"[114336, OMS - DA4570A FILTRO AIRE 2_ DONSSON - PERKINS, CATERPILLAR]",iniciar,0.0,2025-10-27 20:01:17,BLANCA CECILIA AVELLA ARIAS,MO13308,INYECCION,025 DA4570A,DAB14570025,OMS,2025-10-27 15:01:17,True
97946,2025-10-27 17:36:08,0.0,2025-10-27 17:36:08,2025-10-27 12:36:08,"[114312, ENTUBADORA 3 - DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.]",iniciar,0.0,2025-10-27 17:36:08,BLANCA DORA CUBIDES SANCHEZ,MO13393,CORTE,025 DA2570,DAB02570025,ENTUBADORA 3,2025-10-27 12:36:08,True
98067,2025-10-27 19:24:59,0.0,2025-10-27 19:24:59,2025-10-27 14:24:59,"[114330, EMBALAJE - DA2783 FILTRO AIRE DONSSON- CHEV.FRR, FTR, HITACHI]",iniciar,0.0,2025-10-27 19:24:59,CLARA MARLEN OSPINA CASTRO,MO13360,EMBALAJE,025 DA2783,DAB02783025,EMBALAJE,2025-10-27 14:24:59,True
97988,2025-10-27 18:25:57,0.0,2025-10-27 18:25:57,2025-10-27 13:25:57,"[114308, PLISADORA - DA2976 FILTRO AIRE MACK GRANITE]",iniciar,0.0,2025-10-27 18:25:57,DANILO PUPO SIERRA,MO13374,PLISADO,025 DA2976,DAB02976025,PLISADORA,2025-10-27 13:25:57,True
97970,2025-10-27 18:15:01,0.0,2025-10-27 18:15:01,2025-10-27 13:15:01,"[114320, TECNONEXT NARANJA - DA2123 FILTRO AIRE MACK]",iniciar,0.0,2025-10-27 18:15:01,DAVID ESTEBAN MALAGON CETINA,MO13357,INYECCION,025 DA2123,DAR02123025,TECNONEXT NARANJA,2025-10-27 13:15:01,True
98029,2025-10-27 19:10:02,0.0,2025-10-27 19:10:02,2025-10-27 14:10:02,"[114313, EMBALAJE - DA2783 FILTRO AIRE DONSSON- CHEV.FRR, FTR, HITACHI]",iniciar,0.0,2025-10-27 19:10:02,DEICY MILENA LOPEZ VELASQUEZ,MO13360,EMBALAJE,025 DA2783,DAB02783025,EMBALAJE,2025-10-27 14:10:02,True
97804,2025-10-27 13:16:06,0.0,2025-10-27 13:16:06,2025-10-27 08:16:06,"[114262, HENNECKE - DA8182 FILTRO AIRE GENERADORES BAUDOUIN 898HP]",iniciar,0.0,2025-10-27 13:16:06,DICZON CAMILO REY RUIZ,MO13279,INYECCION,025 DA8182,DAB08182025,HENNECKE,2025-10-27 08:16:06,True
97806,2025-10-27 13:16:37,0.0,2025-10-27 13:16:37,2025-10-27 08:16:37,"[114263, PLISADORA - DA8234 FILTRO AIRE - DONGFENG]",iniciar,0.0,2025-10-27 13:16:37,EDINSON PUPO SIERRA,MO13370,PLISADO,025 DA8234,DAB08234025,PLISADORA,2025-10-27 08:16:37,True
98073,2025-10-27 19:39:52,0.0,2025-10-27 19:39:52,2025-10-27 14:39:52,"[114332, CONTROL DE CALIDAD - DA2783 FILTRO AIRE DONSSON- CHEV.FRR, FTR, HITACHI]",iniciar,0.0,2025-10-27 19:39:52,EDUARDO ALONSO CARDENAS MURILLO,MO13360,EMBALAJE,025 DA2783,DAB02783025,CONTROL DE CALIDAD,2025-10-27 14:39:52,True


In [11]:
#def_op.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/tiempo_operarios.xlsx")

## UNIR DATAFRAMES

In [ ]:
#df_union.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/control_operarios.xlsx")